In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

base = "doge"
quote = "usdt"

enable_logging = True

symbol = f"{base.upper()}{quote.upper()}"

start_date_str = "2023-08-01T00:00:00Z"
end_date_str = "2023-08-31T23:59:59Z"
start_date = datetime.fromisoformat(start_date_str.replace("Z", "+00:00"))
end_date = datetime.fromisoformat(end_date_str.replace("Z", "+00:00"))

interval = "1m"
limit = 1000
delta_hours = 12

url = "https://api.binance.com/api/v3/klines"
fetch_delay_sec = 0.5

all_data = []

while start_date < end_date:
    batch_end_date = start_date + timedelta(hours=delta_hours)
    if batch_end_date > end_date:
        batch_end_date = end_date
    
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": int(start_date.timestamp() * 1000),
        "endTime": int(batch_end_date.timestamp() * 1000),
        "limit": limit
    }

    while True:
        response = requests.get(url, params=params)
        data = response.json()

        if not data:
            break

        all_data.extend(data)

        if enable_logging:
            print(f"Fetched {len(data)} records from {start_date.strftime('%Y-%m-%d %H:%M:%S')}")

        last_timestamp = data[-1][0]
        next_start_time = datetime.fromtimestamp(last_timestamp / 1000, tz=start_date.tzinfo) + timedelta(minutes=1)

        if next_start_time >= batch_end_date:
            break

        params["startTime"] = int(next_start_time.timestamp() * 1000)
    
    if fetch_delay_sec > 0:
        time.sleep(fetch_delay_sec)
    
    start_date = batch_end_date

df = pd.DataFrame(all_data, columns=[
    "timestamp", "open", "high", "low", "close", "volume", "close_time",
    "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume",
    "taker_buy_quote_asset_volume", "ignore"
])

df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms', utc=True)

file_name = f"{base.upper()}-{quote.upper()}_{start_date_str[:10]}_to_{end_date_str[:10]}_{interval}.csv"
df.to_csv(file_name, index=False)

num_lines = len(df)
report = f"Data saved to {file_name} with {num_lines} lines extracted."

if enable_logging:
    print(report)

Fetched 721 records from 2023-08-01 00:00:00
Fetched 721 records from 2023-08-01 12:00:00
Fetched 721 records from 2023-08-02 00:00:00
Fetched 721 records from 2023-08-02 12:00:00
Fetched 721 records from 2023-08-03 00:00:00
Fetched 721 records from 2023-08-03 12:00:00
Fetched 721 records from 2023-08-04 00:00:00
Fetched 721 records from 2023-08-04 12:00:00
Fetched 721 records from 2023-08-05 00:00:00
Fetched 721 records from 2023-08-05 12:00:00
Fetched 721 records from 2023-08-06 00:00:00
Fetched 721 records from 2023-08-06 12:00:00
Fetched 721 records from 2023-08-07 00:00:00
Fetched 721 records from 2023-08-07 12:00:00
Fetched 721 records from 2023-08-08 00:00:00
Fetched 721 records from 2023-08-08 12:00:00
Fetched 721 records from 2023-08-09 00:00:00
Fetched 721 records from 2023-08-09 12:00:00
Fetched 721 records from 2023-08-10 00:00:00
Fetched 721 records from 2023-08-10 12:00:00
Fetched 721 records from 2023-08-11 00:00:00
Fetched 721 records from 2023-08-11 12:00:00
Fetched 72

In [13]:
df.head(3)

,timestamp,base,quote,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2023-07-01 00:00:00+00:00,DOGE,USDT,0.06646000,0.06658000,0.06645000,0.06655000,1066058.00000000,1688169659999,70921.71967000,104,554772.00000000,36898.40767000,0
1,2023-07-01 00:01:00+00:00,DOGE,USDT,0.06655000,0.06655000,0.06645000,0.06646000,562796.00000000,1688169719999,37417.36913000,66,243819.00000000,16212.07310000,0
2,2023-07-01 00:02:00+00:00,DOGE,USDT,0.06645000,0.06665000,0.06645000,0.06657000,2623481.00000000,1688169779999,174640.09056000,269,1894869.00000000,126091.95575000,0
